In [1]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    GOOGLE_API_KEY: str
    model_config = SettingsConfigDict(env_file=".env")

env = Settings()

In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_2 = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=env.GOOGLE_API_KEY)

In [27]:
from qdrant_client.http.models import Distance

collection_name = "doctors_packages"
dimension = 768
distance = Distance.COSINE

In [28]:
import json

with open("doctors_final.json", "r") as f:
    doct_data = json.load(f)

print(doct_data[0])

{'id': '5eae0017-40dd-4961-869f-79d9e45d87f2', 'name': 'Adventia Emilia Krysna Sipi Seda, M.M., M.Psi., Psikolog', 'specialization_name': 'Psikologi', 'specialization_name_en': 'Psychology', 'sub_specialization_name': 'Psikolog', 'sub_specialization_name_en': 'Psychologist', 'hospital_name': 'Siloam Hospitals Yogyakarta'}


In [29]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

In [30]:
from qdrant_client.http.models import VectorParams

if(client.collection_exists(collection_name=collection_name) == False):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=dimension, distance=distance),
    )

In [31]:
import uuid
from qdrant_client.http.models import PointStruct

for row in doct_data:
    text = f"Doctors Name: {row['name']}, Specialization: {row['specialization_name']}, Sub-Specialization: {row['sub_specialization_name']}"f"Doctors Name: {row['name']}, Specialization: {row['specialization_name']}, Sub-Specialization: {row['sub_specialization_name']}"f"Doctors Name: {row['name']}, Specialization: {row['specialization_name']}, Sub-Specialization: {row['sub_specialization_name']}"
    emb = embeddings.embed_query(text)
    client.upsert(
        collection_name=collection_name,
        points=[
            PointStruct(
                id=str(uuid.uuid4()),
                vector=emb, 
                payload={
                    "page_content": text,
                    "metadata": {
                        "id": row['id'],
                        "name": row['name'],
                        "specialization_name": row['specialization_name'],
                        "sub_specialization_name": row['sub_specialization_name'],
                    },
                },
            )
        ],
    )
    print(text)


Doctors Name: Adventia Emilia Krysna Sipi Seda, M.M., M.Psi., Psikolog, Specialization: Psikologi, Sub-Specialization: PsikologDoctors Name: Adventia Emilia Krysna Sipi Seda, M.M., M.Psi., Psikolog, Specialization: Psikologi, Sub-Specialization: PsikologDoctors Name: Adventia Emilia Krysna Sipi Seda, M.M., M.Psi., Psikolog, Specialization: Psikologi, Sub-Specialization: Psikolog
Doctors Name: Dokter Umum Siloam Yogyakarta, Specialization: Kedokteran Umum, Sub-Specialization: Dokter UmumDoctors Name: Dokter Umum Siloam Yogyakarta, Specialization: Kedokteran Umum, Sub-Specialization: Dokter UmumDoctors Name: Dokter Umum Siloam Yogyakarta, Specialization: Kedokteran Umum, Sub-Specialization: Dokter Umum
Doctors Name: dr. Andrean Jefrian Manihuruk, Specialization: Kedokteran Umum, Sub-Specialization: Dokter UmumDoctors Name: dr. Andrean Jefrian Manihuruk, Specialization: Kedokteran Umum, Sub-Specialization: Dokter UmumDoctors Name: dr. Andrean Jefrian Manihuruk, Specialization: Kedokteran 

In [32]:
from langchain_qdrant import QdrantVectorStore
def get_retriever():

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings_2,
    )
    
    return vector_store.as_retriever()

In [33]:
from langchain_core.tools import tool
from typing import Annotated, List

@tool
def search_doctor(query: Annotated[str, "search query must contain keywords related to Doctors packages"]) -> List[str]:
    """
    Search for doctors packages using the provided query keywords.
    Returns a list of page contents from the retrieved results.
    """
    retriever = get_retriever()
    results = retriever.invoke(query, k=10)
    return [result.page_content for result in results]

In [34]:
def get_retriever():
    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings,  # use FastEmbedEmbeddings
    )
    return vector_store.as_retriever()

In [35]:
# access the Google Gemini API
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=env.GOOGLE_API_KEY,
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that provides information about Siloam hospitals."),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

In [36]:
chain.invoke({"question": "Saya mau cek gula darah di siloam, ada dokter apa aja ya?"})

AIMessage(content='Untuk pemeriksaan gula darah di Siloam Hospitals, Anda dapat berkonsultasi dengan dokter umum atau dokter spesialis penyakit dalam (internist). Dokter umum dapat melakukan pemeriksaan awal dan memberikan saran terkait kadar gula darah Anda. Jika diperlukan penanganan lebih lanjut atau terdapat kondisi medis lain yang terkait, dokter umum dapat merujuk Anda ke dokter spesialis penyakit dalam.\n\nUntuk mengetahui jadwal dan ketersediaan dokter di Siloam Hospitals terdekat, Anda dapat mengunjungi website resmi Siloam Hospitals atau menghubungi call center mereka. Anda juga dapat menggunakan aplikasi MySiloam untuk membuat janji temu dengan dokter.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e03525da-8528-41c9-b31f-b55d07553d55-0', usage_metadata={'input_tokens': 28, 'output_tokens': 114, 'total_tokens': 142, 'input_token

In [37]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
# Define state for application
class State(TypedDict):
    question: str
    context: List[str]
    search: str
    answer: str

In [81]:
def get_context(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
                You are an expert in finding doctors at Siloam hospitals.
                Based on the user's illness and specification in the question, provide relevant keywords for searching the doctor's name and specialization.
                Only return one keyword in English, related to the illness or specialization.
                Do not provide any other information.
            """),
            ("human", "{question}"),
        ]
    )
    chain = prompt | llm
    result = chain.invoke({"question": state["question"]})
    return {"search": result.content}

In [77]:
def retrieve(state: State):
    retrieved_docs = search_doctor(state["search"])
    return {"context": retrieved_docs}

In [82]:
def generate(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
                You are an assistant that provides information about doctors and the doctors specialization at Siloam hospitals.
                You will generate a response based on the context provided.
                The response should be concise and relevant to the question asked.
                Please include doctors' names and their specializations from the context.
                package list knowledge: 
                {context}
                If the context is empty, you can provide a general response about MCU packages.
                Please always include related packages in your response.
            """),
            ("human", "{question}"),
        ]
    )
    chain = prompt | llm
    result = chain.invoke({"question": state["question"], "context": state["context"]})
    return {"answer": result.content}

In [79]:
graph_builder = StateGraph(State).add_sequence([get_context, retrieve, generate])
graph_builder.add_edge(START, "get_context")
graph = graph_builder.compile()

In [90]:
response = graph.invoke({
	"question": "Saya sakit jantung, dokter siapa saja yang tersedia ?",
	"context": [],
	"search": "",
	"answer": ""
})
print(response["answer"])

Berikut dokter spesialis jantung yang tersedia di Siloam Hospitals:
1. dr. Benny Mulyanto Setiadi, SpJP(K), FIHA dengan subspesialis Aritmia
2. dr. Andi Alief Utama Armyn, SpJP (K) dengan subspesialis Kardiologi Pediatrik dan Penyakit Jantung Bawaan
3. Dr. dr. Saskia Dyah Handari, SpJP (K), FESC, FAsCC, FASE, FIHA dengan subspesialis Ekokardiografi
